# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data = pd.read_csv("../output_data/Weather.csv")
city_data = city_data.drop(columns=["Unnamed: 0"])
city_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Busselton,-33.65,115.33,57.99,84,83,4.56,AU,1592534595
1,Vaini,-21.20,-175.20,73.40,88,75,10.29,TO,1592534615
2,Clyde River,70.47,-68.59,32.00,92,90,10.29,CA,1592534629
3,Hithadhoo,-0.60,73.08,84.65,73,74,3.38,MV,1592534598
4,Ancud,-41.87,-73.82,48.00,87,75,9.17,CL,1592534597
...,...,...,...,...,...,...,...,...,...
560,Hofn,64.25,-15.21,49.01,83,0,0.51,IS,1592534590
561,Tambura,5.60,27.47,67.21,98,100,4.68,SS,1592535086
562,Napanee,44.25,-76.95,71.60,64,75,5.82,CA,1592534885
563,Toora-Khem,52.47,96.11,65.34,51,20,7.45,RU,1592535086


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# configure gmaps
gmaps.configure(api_key = g_key)
locations = city_data[["Lat", "Lng"]]
humidity = city_data["Humidity"].astype(float)

In [4]:
# Create a Heatmap layer
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=np.max(humidity),
                                 point_radius = 1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
narrow_city_data = city_data.loc[(city_data["Max Temp"] > 70) & (city_data["Max Temp"] < 80) & 
                                 (city_data["Wind Speed"] < 10)  & (city_data["Cloudiness"] == 0)].dropna()
narrow_city_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,Havelock,34.88,-76.90,75.99,85,0,1.01,US,1592534489
44,Chitral,35.85,71.79,75.81,21,0,1.34,PK,1592535011
75,Ulaangom,49.98,92.07,71.35,30,0,7.43,MN,1592535015
137,Murdochville,48.96,-65.50,74.21,44,0,9.73,CA,1592535024
152,Yangi Marg`ilon,40.43,71.72,71.60,30,0,2.24,UZ,1592534913
251,Jalu,29.03,21.55,73.06,31,0,4.54,LY,1592534655
280,Aginskoye,51.10,114.53,74.71,42,0,0.92,RU,1592535045
312,Karatau,43.17,70.47,74.98,20,0,3.27,KZ,1592534844
434,Datong,40.09,113.29,75.58,42,0,4.47,CN,1592535067
463,Sabzawār,36.21,57.68,79.20,17,0,8.97,IR,1592535071


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = narrow_city_data.loc[:,["Country","City", "Lat", "Lng"]]
hotel_df["Hotel Name"]= ""
hotel_df

,Country,City,Lat,Lng,Hotel Name
11,US,Havelock,34.88,-76.90,
44,PK,Chitral,35.85,71.79,
75,MN,Ulaangom,49.98,92.07,
137,CA,Murdochville,48.96,-65.50,
152,UZ,Yangi Marg`ilon,40.43,71.72,
251,LY,Jalu,29.03,21.55,
280,RU,Aginskoye,51.10,114.53,
312,KZ,Karatau,43.17,70.47,
434,CN,Datong,40.09,113.29,
463,IR,Sabzawār,36.21,57.68,


In [7]:
params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city= row["City"]
    params["location"] = f"{lat},{lng}"
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(url, params=params).json()
    results = response['results']
    try:
        print(f"Closest hotel in {city} is: {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print("Couldnt find hotel. Skipping...")
        hotel_df.loc[index, "Hotel Name"] = ""

    print("-----------------------------")
print("Data Retrieval Complete")      
print("-----------------------------")

Retrieving Results for Index 11: Havelock.
Closest hotel in Havelock is: Hampton Inn Havelock.
-----------------------------
Retrieving Results for Index 44: Chitral.
Closest hotel in Chitral is: Al Mughal Hotel.
-----------------------------
Retrieving Results for Index 75: Ulaangom.
Closest hotel in Ulaangom is: Grand restaurant & hotel.
-----------------------------
Retrieving Results for Index 137: Murdochville.
Closest hotel in Murdochville is: Les Suites économique Murdochville.
-----------------------------
Retrieving Results for Index 152: Yangi Marg`ilon.
Closest hotel in Yangi Marg`ilon is: Hotel Mehmon Saroy.
-----------------------------
Retrieving Results for Index 251: Jalu.
Closest hotel in Jalu is: Jalu Hotel.
-----------------------------
Retrieving Results for Index 280: Aginskoye.
Closest hotel in Aginskoye is: Gostinitsa Gobi.
-----------------------------
Retrieving Results for Index 312: Karatau.
Couldnt find hotel. Skipping...
-----------------------------
Retrie

In [8]:
hotel_df = hotel_df[hotel_df["Hotel Name"]!='']
hotel_df

,Country,City,Lat,Lng,Hotel Name
11,US,Havelock,34.88,-76.90,Hampton Inn Havelock
44,PK,Chitral,35.85,71.79,Al Mughal Hotel
75,MN,Ulaangom,49.98,92.07,Grand restaurant & hotel
137,CA,Murdochville,48.96,-65.50,Les Suites économique Murdochville
152,UZ,Yangi Marg`ilon,40.43,71.72,Hotel Mehmon Saroy
251,LY,Jalu,29.03,21.55,Jalu Hotel
280,RU,Aginskoye,51.10,114.53,Gostinitsa Gobi
434,CN,Datong,40.09,113.29,Datong Weidu International Hotel
463,IR,Sabzawār,36.21,57.68,Sabzevar Tourism hotel
475,DO,La Romana,18.43,-68.97,Hotel River View


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))